# 📚 Data

This notebook contains code for the data in this experiment suite.

## Setup 

In [1]:
import autorootcwd

In [ ]:
from typing import Dict, Tuple, Any
from tqdm import tqdm

import pandas as pd
from transformers import AutoTokenizer
from datasets import DatasetDict, Dataset, load_dataset
from src.utils import format_int

## Memorizing Dataset

We will create a single sample dummy dataset to test that we can overfit a single sample.


In [ ]:
# Create a dataset with a single example
sentence = "I am a large language model and I can memorize this sentence."
dataset = Dataset.from_dict({"text": [sentence]})
memorize = DatasetDict({"train": dataset, "validation": dataset, "test": dataset})

print(f"Created a dataset with a single example in train, validation and test: {sentence}")

In [ ]:
# Push to Hugging Face Hub
repo_name = "memorize"
memorize.push_to_hub(repo_name)

print(f"Pushed to https://huggingface.co/datasets/mikasenghaas/{repo_name}")

## WikiText 2

For now, we will usoe a tiny dataset `Salesforce/wikitext/wikitext-2-raw-v1`. It has a train, validation and test split that consist of 37K, 1.8K and 2.2K examples respectively.

In [ ]:
# Load WikiText 2
wiki = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1", cache_dir="/workspace/huggingface")
train_wiki, val_wiki, test_wiki = wiki["train"], wiki["validation"], wiki["test"]

print(f"Loaded {len(train_wiki)/1e3:.1f}K training, {len(val_wiki)/1e3:.1f}K validation and {len(test_wiki)/1e3:.1f}K test examples.")

A single example just has a `text` field, which contains a single line of text. They are parsed from high quality Wikipedia articles. We can already see that there are loads of empty lines and other artiffacts like headlines.

In [ ]:
# Examples
for example in train_wiki.take(5):
    print(example)

We are going to remove empty lines, headlines, and trailing whitespace.

In [ ]:
def non_empty_text(examples: Dict[str, Any]) -> bool:
    return examples["text"] != ""

def non_headline(examples: Dict[str, Any]) -> bool:
    return not examples["text"].startswith(" = ")

def strip_headline(examples: Dict[str, Any]) -> Dict[str, Any]:
    examples["text"] = examples["text"].lstrip().rstrip()
    return examples

In [ ]:
train_wiki_processed = train_wiki.filter(non_empty_text).filter(non_headline).map(strip_headline)
val_wiki_processed = val_wiki.filter(non_empty_text).filter(non_headline).map(strip_headline)
test_wiki_processed = test_wiki.filter(non_empty_text).filter(non_headline).map(strip_headline)

print(f"Processed {len(train_wiki_processed)/1e3:.1f}K training, {len(val_wiki_processed)/1e3:.1f}K validation and {len(test_wiki_processed)/1e3:.1f}K test examples.")

In [ ]:
for example in train_wiki_processed.take(5):
    print(example)

Looks good! Let's get some statistics on the processed dataset.

In [ ]:
# Dataset statistics
get_num_examples = lambda dataset: len(dataset)
get_num_tokens = lambda dataset, tokenizer: sum(len(tokenizer.encode(example['text'])) for example in dataset)

# Llama 2 tokenizer
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
llama3_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")

stats = pd.DataFrame({
    'Split': ['Train', 'Validation', 'Test'],
    'Examples': map(format_int, [get_num_examples(train_wiki_processed), get_num_examples(val_wiki_processed), get_num_examples(test_wiki_processed)]),
    'GPT-2 Tokens': map(format_int, [get_num_tokens(train_wiki_processed, gpt2_tokenizer), get_num_tokens(val_wiki_processed, gpt2_tokenizer), get_num_tokens(test_wiki_processed, gpt2_tokenizer)]),
    'Llama-3 Tokens': map(format_int, [get_num_tokens(train_wiki_processed, llama3_tokenizer), get_num_tokens(val_wiki_processed, llama3_tokenizer), get_num_tokens(test_wiki_processed, llama3_tokenizer)])
}).set_index('Split')

stats

Finally, let's push the processed datasets to the Hugging Face Hub.

In [ ]:
# Push to Hugging Face Hub
data = DatasetDict({
    'train': train_wiki_processed,
    'validation': val_wiki_processed,
    'test': test_wiki_processed
})

repo_name = "wikitext-2"
data.push_to_hub(repo_name)

print(f"Pushed to https://huggingface.co/datasets/mikasenghaas/{repo_name}")

## FinewebEdu

The [FinewebEdu](https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu) dataset is a large-scale pre-training dataset developed by the Hugging Face team. The smaller version consists of 1.3T high-quality tokens that have been filtered for quality using Llama 2 70B

We are going to use the 10BT version of the dataset which contains 9.67M samples, corresponding to roughly 10B GPT-2 training tokens.

In [ ]:
# Load FinewebEdu (10BT)
finewebedu_10bt = load_dataset("HuggingFaceFW/fineweb-edu", "sample-10BT", split="train", cache_dir="/workspace/huggingface")

print(f"Loaded {len(finewebedu_10bt)/1e6:.1f}M training examples.")

In [ ]:
# Randomly sample 10% of FinewebEdu 10BT (1BT)
finewebedu_1bt = finewebedu_10bt.shuffle(seed=42).select(range(int(len(finewebedu_10bt) * 0.1)))

print(f"Sampled {len(finewebedu_1bt)/1e6:.1f}M training examples (10% of 10BT).")

In [ ]:
# Randomly sample 10% of FinewebEdu 1.25BT
finewebedu_100mt = finewebedu_10bt.shuffle(seed=42).select(range(int(len(finewebedu_1bt) * 0.1)))

print(f"Sampled {len(finewebedu_100mt)/1e6:.1f}M training examples (10% of 1BT).")

In [ ]:
# Dataset statistics
get_num_examples = lambda dataset: len(dataset)
get_num_tokens = lambda dataset, tokenizer: sum(len(tokenizer.encode(example['text'])) for example in dataset)

# GPT-2 tokenizer
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")
llama3_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# Function to calculate average tokens per example
def calc_avg_tokens(dataset, tokenizer, num_samples=10000):
    subset_data = dataset.shuffle(seed=42).select(range(num_samples))
    return get_num_tokens(subset_data, tokenizer) / num_samples

# Calculate average tokens for each dataset
datasets = {
    '10BT': finewebedu_10bt,
    '1BT': finewebedu_1bt,
    '100MT': finewebedu_100mt
}

stats_data = []
for name, dataset in datasets.items():
    num_examples = get_num_examples(dataset)
    avg_gpt2_tokens = calc_avg_tokens(dataset, gpt_tokenizer)
    avg_llama3_tokens = calc_avg_tokens(dataset, llama3_tokenizer)
    
    stats_data.append({
        'Dataset': name,
        'Examples': format_int(num_examples),
        'GPT-2 Tokens': format_int(num_examples * avg_gpt2_tokens),
        'Llama-3 Tokens': format_int(num_examples * avg_llama3_tokens)
    })

stats = pd.DataFrame(stats_data).set_index(['Dataset'])
stats

Nice, we are getting 100%, 10%, and 1% of the dataset, i.e. roughly 10B, 1B, and 100M GPT-2 training tokens, respectively. Let's upload the processed dataset to the Hugging Face Hub.

In [ ]:
# Upload to Hugging Face Hub
repo_name = "fineweb-edu-10bt"
finewebedu_10bt.push_to_hub(repo_name)

print(f"Pushed to https://huggingface.co/datasets/mikasenghaas/{repo_name}")

In [ ]:
# Upload to Hugging Face Hub
repo_name = "fineweb-edu-1bt"
finewebedu_1bt.push_to_hub(repo_name)

print(f"Pushed to https://huggingface.co/datasets/mikasenghaas/{repo_name}")

In [ ]:
# Upload to Hugging Face Hub
repo_name = "fineweb-edu-100mt"
finewebedu_100mt.push_to_hub(repo_name)

print(f"Pushed to https://huggingface.co/datasets/mikasenghaas/{repo_name}")